This notebook provides examples to go along with the [textbook](https://underactuated.csail.mit.edu/acrobot.html).  I recommend having both windows open, side-by-side!


In [ ]:
import matplotlib.pyplot as plt
import mpld3
import numpy as np
from IPython.display import HTML, display
from pydrake.all import (
    DiagramBuilder,
    FirstOrderLowPassFilter,
    Gain,
    LinearQuadraticRegulator,
    LogVectorOutput,
    Multiplexer,
    RandomDistribution,
    RandomSource,
    Simulator,
)

from underactuated import running_as_notebook
from underactuated.quadrotor2d import Quadrotor2D, Quadrotor2DVisualizer

if running_as_notebook:
    mpld3.enable_notebook()

# LQR for the Planar Quadrotor w/ "Wind"

Let's first consider the planar quadrotor with a colored noise force (wind) disturbance, but with LQR that completely ignores the wind.

In [ ]:
def planar_quadrotor_example(animate=False):
    def QuadrotorLQR(plant):
        context = plant.CreateDefaultContext()
        context.SetContinuousState(np.zeros([6, 1]))
        plant.get_input_port(0).FixValue(
            context, plant.mass * plant.gravity / 2.0 * np.array([1, 1])
        )

        Q = np.diag([10, 10, 10, 1, 1, (plant.length / 2.0 / np.pi)])
        R = np.array([[0.1, 0.05], [0.05, 0.1]])

        return LinearQuadraticRegulator(plant, context, Q, R)

    builder = DiagramBuilder()
    plant = builder.AddSystem(Quadrotor2D())

    controller = builder.AddSystem(QuadrotorLQR(plant))
    builder.Connect(controller.get_output_port(0), plant.GetInputPort("u"))
    builder.Connect(plant.get_output_port(), controller.get_input_port(0))

    # Add simple wind model
    wind = builder.AddSystem(RandomSource(RandomDistribution.kGaussian, 2, 0.1))
    gain = builder.AddSystem(Gain(2, 2))
    filter = builder.AddSystem(FirstOrderLowPassFilter(time_constant=2, size=2))
    builder.Connect(wind.get_output_port(0), gain.get_input_port(0))
    builder.Connect(gain.get_output_port(0), filter.get_input_port(0))
    builder.Connect(filter.get_output_port(0), plant.GetInputPort("w"))

    logger = LogVectorOutput(plant.get_output_port(), builder, 0.1)

    if animate:
        # Setup visualization
        visualizer = builder.AddSystem(Quadrotor2DVisualizer(show=False))
        builder.Connect(plant.get_output_port(0), visualizer.get_input_port(0))

    diagram = builder.Build()

    # Set up a simulator to run this diagram
    simulator = Simulator(diagram)
    context = simulator.get_mutable_context()

    # Simulate
    duration = 10.0 if running_as_notebook else 0.1
    plant_context = plant.GetMyMutableContextFromRoot(context)
    plant_context.SetContinuousState(np.array([1.0, 0.0, 0.2, 0.0, 0.0, 0.0]))
    if animate:
        visualizer.start_recording()
        print("simulating...")
    simulator.AdvanceTo(duration)

    log = logger.FindLog(context)
    fig, ax = plt.subplots(3, 1)
    state_names = ["x", "y", "θ", "ẋ", "ẏ", "θ̇"]
    for i in range(3):
        ax[i].plot(log.sample_times(), log.data()[i, :].T)
        ax[i].set_ylabel(state_names[i])
        ax[i].set_ylim([-0.25, 0.25])
    ax[2].set_xlabel("time (s)")
    display(mpld3.display())

    if animate:
        print("done.\ngenerating animation...")
        ani = visualizer.get_recording_as_animation()
        display(HTML(ani.to_jshtml()))


planar_quadrotor_example(animate=True)

# LQR including the whitening filter

We'll do _almost_ the same experiment again now, but this time we'll also include the state of the wind in the LQR controller.

In [ ]:
def planar_quadrotor_example(animate=False):
    def MakeQuadrotorPlusWind():
        builder = DiagramBuilder()
        plant = builder.AddSystem(Quadrotor2D())
        builder.ExportInput(plant.GetInputPort("u"), "u")
        gain = builder.AddSystem(Gain(2, 2))
        filter = builder.AddSystem(FirstOrderLowPassFilter(time_constant=2, size=2))
        builder.Connect(gain.get_output_port(0), filter.get_input_port(0))
        builder.Connect(filter.get_output_port(0), plant.GetInputPort("w"))
        builder.ExportInput(gain.get_input_port(0), "w")
        mux = builder.AddNamedSystem("mux", Multiplexer([6, 2]))
        builder.Connect(plant.get_output_port(), mux.get_input_port(0))
        builder.Connect(filter.get_output_port(0), mux.get_input_port(1))
        builder.ExportOutput(mux.get_output_port(), "quadrotor_plus_wind_state")
        builder.ExportOutput(plant.get_output_port(), "quadrotor_state")
        return builder.Build()

    def MakeLQR(system):
        context = system.CreateDefaultContext()
        context.SetContinuousState(np.zeros([8, 1]))
        plant = system.GetSubsystemByName("Quadrotor2D")
        # Set the nominal input, u0
        system.GetInputPort("u").FixValue(
            context, plant.mass * plant.gravity / 2.0 * np.array([1, 1])
        )
        system.GetInputPort("w").FixValue(context, [0, 0])

        Q = np.diag([10, 10, 10, 10, 10, 1, 1, (plant.length / 2.0 / np.pi)])
        R = np.array([[0.1, 0.05], [0.05, 0.1]])

        return LinearQuadraticRegulator(system, context, Q, R)

    builder = DiagramBuilder()
    quad_plus_wind = builder.AddSystem(MakeQuadrotorPlusWind())
    quad = quad_plus_wind.GetSubsystemByName("Quadrotor2D")

    controller = builder.AddSystem(MakeLQR(quad_plus_wind))
    builder.Connect(controller.get_output_port(0), quad_plus_wind.GetInputPort("u"))
    builder.Connect(
        quad_plus_wind.GetOutputPort("quadrotor_plus_wind_state"),
        controller.get_input_port(0),
    )

    # Add simple wind model
    wind = builder.AddSystem(RandomSource(RandomDistribution.kGaussian, 2, 0.1))
    builder.Connect(wind.get_output_port(0), quad_plus_wind.GetInputPort("w"))

    logger = LogVectorOutput(
        quad_plus_wind.GetOutputPort("quadrotor_plus_wind_state"), builder, 0.1
    )

    if animate:
        # Setup visualization
        visualizer = builder.AddSystem(Quadrotor2DVisualizer(show=False))
        builder.Connect(
            quad_plus_wind.GetOutputPort("quadrotor_state"),
            visualizer.get_input_port(0),
        )

    diagram = builder.Build()

    # Set up a simulator to run this diagram
    simulator = Simulator(diagram)
    context = simulator.get_mutable_context()

    # Simulate
    duration = 10.0 if running_as_notebook else 0.1
    quad_context = quad.GetMyMutableContextFromRoot(context)
    quad_context.SetContinuousState(np.array([1, 0, 0.2, 0, 0, 0]))
    if animate:
        visualizer.start_recording()
        print("simulating...")
    simulator.AdvanceTo(duration)

    log = logger.FindLog(context)
    fig, ax = plt.subplots(3, 1)
    state_names = ["x", "y", "θ", "ẋ", "ẏ", "θ̇"]
    for i in range(3):
        ax[i].plot(log.sample_times(), log.data()[i, :].T)
        ax[i].set_ylabel(state_names[i])
        ax[i].set_ylim([-0.25, 0.25])
    ax[2].set_xlabel("time (s)")
    display(mpld3.display())

    if animate:
        print("done.\ngenerating animation...")
        ani = visualizer.get_recording_as_animation()
        display(HTML(ani.to_jshtml()))


planar_quadrotor_example(animate=True)